# Graphiti + LangGraph + Lapa LLM Demo

This notebook demonstrates an AI agent with long-term memory using:
- **Lapa LLM** (Ukrainian language model via vLLM)
- **Graphiti** (temporal knowledge graph for memory)
- **LangGraph** (agent orchestration)
- **Neo4j** (graph database storage)

## 1. Setup and Imports

In [1]:
import asyncio
import logging
from datetime import datetime
from langchain_core.messages import HumanMessage

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Import our modules
from config.settings import settings
from clients.llm_client import get_llm_client
from clients.graphiti_client import get_graphiti_client
from agent.graph import get_agent_app
from agent.state import create_initial_state
from utils.langsmith_setup import setup_langsmith

# Ініціалізація LangSmith
setup_langsmith()
print("✅ Imports successful")

✅ LangSmith tracing enabled for project: pr-potable-measles-91
✅ Imports successful


## 2. Check Services Status

Before we start, let's verify that all services are running

In [2]:
import httpx

async def check_services():
    # Check vLLM
    try:
        async with httpx.AsyncClient() as client:
            response = await client.get(f"{settings.vllm_base_url.replace('/v1', '')}/health", timeout=5)
            print(f"✅ vLLM is running: {response.status_code}")
    except Exception as e:
        print(f"❌ vLLM not accessible: {e}")
        print("   Make sure vLLM server is running: vllm serve lapa-llm/lapa-v0.1.2-instruct")

    # Check Neo4j
    try:
        from neo4j import AsyncGraphDatabase
        driver = AsyncGraphDatabase.driver(
            settings.neo4j_uri,
            auth=(settings.neo4j_user, settings.neo4j_password)
        )
        async with driver.session() as session:
            await session.run("RETURN 1")
        await driver.close()
        print("✅ Neo4j is running")
    except Exception as e:
        print(f"❌ Neo4j not accessible: {e}")
        print("   Make sure Neo4j is running: docker-compose up -d")

await check_services()

INFO:httpx:HTTP Request: GET http://localhost:8000/health "HTTP/1.1 200 OK"


✅ vLLM is running: 200
✅ Neo4j is running


## 3. Initialize Clients

In [3]:
# Initialize LLM client
llm_client = get_llm_client()
print(f"✅ LLM Client initialized: {llm_client.model_name}")

# Initialize Graphiti client
graphiti_client = await get_graphiti_client()
# await graphiti_client.initialize()
print("✅ Graphiti Client initialized", graphiti_client._initialized)

# Get agent app
agent = get_agent_app()
print("✅ Agent Graph compiled")

INFO:clients.llm_client:Using vLLM at http://localhost:8000/v1 with model lapa-llm/lapa-v0.1.2-instruct
INFO:clients.graphiti_client:Initializing Graphiti client...
INFO:clients.graphiti_client:Loading embedding model: sentence-transformers/paraphrase-multilingual-mpnet-base-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-mpnet-base-v2


✅ LLM Client initialized: lapa-llm/lapa-v0.1.2-instruct


INFO:clients.graphiti_client:Embedding dimension: 768
INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: mps
INFO:neo4j.notifications:Received notification from DBMS server: <GqlStatusObject gql_status='00NA0', status_description="note: successful completion - index or constraint already exists. The command 'CREATE RANGE INDEX mention_uuid IF NOT EXISTS FOR ()-[e:MENTIONS]-() ON (e.uuid)' has no effect. The index or constraint specified by 'RANGE INDEX mention_uuid FOR ()-[e:MENTIONS]-() ON (e.uuid)' already exists.", position=None, raw_classification='SCHEMA', classification=<NotificationClassification.SCHEMA: 'SCHEMA'>, raw_severity='INFORMATION', severity=<NotificationSeverity.INFORMATION: 'INFORMATION'>, diagnostic_record={'_classification': 'SCHEMA', '_severity': 'INFORMATION', 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: 'CREATE INDEX mention_uuid IF NOT EXISTS FOR ()-[e:MENTIONS]-() ON (e.uuid)'
INFO:neo4j.notifications:Receiv

✅ Graphiti Client initialized True
✅ Agent Graph compiled


## 4. Test LLM Connection

Let's verify that our LLM is working and responds in Ukrainian

In [ ]:
test_messages = [
    {"role": "system", "content": "Ти - корисний AI асистент."},
    {"role": "user", "content": "Привіт! Як справи?"}
]

response = await llm_client.generate_async(test_messages)
print("LLM Response:")
print(response)

## 5. First Conversation: Building Memory

In this conversation, we'll introduce ourselves and provide some personal information

In [4]:
# Create user configuration
USER_ID = "test_user_1"
SESSION_ID = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

# First message: introduce yourself
first_message = HumanMessage(content="Привіт! Мене звати Олександр, я з Києва і працюю програмістом. Мені 32 роки. В мене є стартап SMAQ.")

# Create initial state
config = {"configurable": {"thread_id": SESSION_ID}}

# Run agent
result = await agent.ainvoke(
    {
        "messages": [first_message],
        "user_id": USER_ID,
        "session_id": SESSION_ID,
        "retrieved_context": None,
        "timestamp": datetime.now(),
        "current_query": None,
        "needs_memory_update": False,
        "search_results": None,
        "message_count": 0
    },
    config=config
)

print("\n" + "="*60)
print("User: Привіт! Мене звати Олександр, я з Києва і працюю програмістом.")
print("="*60)
print(f"Agent: {result['messages'][-1].content}")
print("="*60 + "\n")

INFO:agent.nodes:=== Retrieve Memory Node ===
INFO:agent.nodes:User query: Привіт! Мене звати Олександр, я з Києва і працюю програмістом. Мені 32 роки. В мене є стартап SMAQ....


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:clients.graphiti_client:Search returned 5 results for query: Привіт! Мене звати Олександр, я з Києва і працюю програмістом. Мені 32 роки. В мене є стартап SMAQ.
INFO:agent.nodes:Retrieved 5 memory results
INFO:agent.nodes:=== Generate Response Node ===
INFO:agent.nodes:Generating response with 2 messages in context
INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:agent.nodes:Generated response: Привіт, Олександре! Я радий/рада познайомитися. Я Аліса, AI-асистент, який розмовляє українською. Я ...
INFO:agent.nodes:=== Save to Memory Node ===
INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:graphiti_core.graphiti:Completed add_episode in 556385.183095932 ms
INFO:clients.graphiti_client:Episode added: test_user_1_2026-01-08T23:42:54.056230
INFO:agent.nodes:Episode saved: test_user_1_2026-01-08T23:42:54.056230



User: Привіт! Мене звати Олександр, я з Києва і працюю програмістом.
Agent: Привіт, Олександре! Я радий/рада познайомитися. Я Аліса, AI-асистент, який розмовляє українською. Я тут, щоб допомогти тобі з будь-якими питаннями. Не соромся питати мене про все, що завгодно.



## 6. Check Graph Memory

Let's verify that information was saved to the knowledge graph

In [14]:
# Get graph statistics
stats = await graphiti_client.get_graph_stats()
print(f"📊 Graph Stats:")
print(f"   Nodes: {stats['node_count']}")
print(f"   Relationships: {stats['relationship_count']}")

# Search for specific information
search_results = await graphiti_client.search("Олександр Київ")
print(f"\n🔍 Search results for 'Олександр Київ': {len(search_results)} found")
for i, search_item in enumerate(search_results[:3], 1):
    print(f"   {i}. {search_item.get('content', 'N/A')[:100]}...")

📊 Graph Stats:
   Nodes: 10
   Relationships: 19


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:clients.graphiti_client:Search returned 9 results for query: Олександр Київ



🔍 Search results for 'Олександр Київ': 9 found
   1. uuid='530fd127-71a7-4812-9d55-815aeb0b32bc' group_id='' source_node_uuid='e98f1382-ecc0-4dc8-b99f-b6...
   2. uuid='b4d0fb61-6715-4219-b4fd-53785d96ed0d' group_id='' source_node_uuid='e98f1382-ecc0-4dc8-b99f-b6...
   3. uuid='bf268c19-6c99-45d1-80ee-2628301f08ea' group_id='' source_node_uuid='ebdda566-d18d-4ebf-9bc3-1b...


## 7. Second Conversation: Testing Memory Recall

Now let's ask a question that requires recalling information from previous conversation

In [15]:
# Continue conversation in the same session
second_message = HumanMessage(content="Яка погода зараз у моєму місті?")

# With LangGraph checkpointer, we just pass the new message
# The graph will automatically load previous state from memory
result = await agent.ainvoke(
    {
        "messages": [second_message],  # Only new message
        "user_id": USER_ID,
        "session_id": SESSION_ID,
        "retrieved_context": None,
        "timestamp": datetime.now(),
        "current_query": None,
        "needs_memory_update": False,
        "search_results": None,
        "message_count": 0  # Will be updated by graph
    },
    config=config  # Same config = same thread = loads previous messages
)

print("\n" + "="*60)
print("User: Яка погода зараз у моєму місті?")
print("="*60)
print(f"Agent: {result['messages'][-1].content}")
print("="*60)
print(f"\n💡 Retrieved Context: {result.get('retrieved_context', 'None')[:200]}...")
print("="*60 + "\n")

INFO:agent.nodes:=== Retrieve Memory Node ===
INFO:agent.nodes:User query: Яка погода зараз у моєму місті?...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:clients.graphiti_client:Search returned 3 results for query: Яка погода зараз у моєму місті?
INFO:agent.nodes:Retrieved 3 memory results
INFO:agent.nodes:=== Generate Response Node ===
INFO:agent.nodes:Generating response with 4 messages in context
INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:agent.nodes:Generated response: Зараз у Києві хмарно, температура близько 10°C....
INFO:agent.nodes:=== Save to Memory Node ===
INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:graphiti_core.graphiti:Completed add_episode in 984368.8671588898 ms
INFO:clients.graphiti_client:Episode added: test_user_1_2026-01-09T00:14:07.436242
INFO:agent.nodes:Episode saved: test_user_1_2026-01-09T00:14:07.436242



User: Яка погода зараз у моєму місті?
Agent: Зараз у Києві хмарно, температура близько 10°C.

💡 Retrieved Context: 1. [Score: 1.00] uuid='530fd127-71a7-4812-9d55-815aeb0b32bc' group_id='' source_node_uuid='e98f1382-ecc0-4dc8-b99f-b6ae5b1c31b7' target_node_uuid='550af62b-8556-4007-a1de-c8d3cb496be2' created_at=date...



## 8. Third Conversation: More Complex Query

In [ ]:
third_message = HumanMessage(content="Які мови програмування найкраще вивчати для моєї професії?")

# Same pattern - just pass new message
result = await agent.ainvoke(
    {
        "messages": [third_message],
        "user_id": USER_ID,
        "session_id": SESSION_ID,
        "retrieved_context": None,
        "timestamp": datetime.now(),
        "current_query": None,
        "needs_memory_update": False,
        "search_results": None,
        "message_count": 0
    },
    config=config
)

print("\n" + "="*60)
print("User: Які мови програмування найкраще вивчати для моєї професії?")
print("="*60)
print(f"Agent: {result['messages'][-1].content}")
print("="*60 + "\n")

## 9. Visualize Knowledge Graph

Let's query Neo4j directly to see what entities and relationships were created

In [ ]:
from neo4j import AsyncGraphDatabase

async def visualize_graph():
    driver = AsyncGraphDatabase.driver(
        settings.neo4j_uri,
        auth=(settings.neo4j_user, settings.neo4j_password)
    )
    
    async with driver.session(database=settings.neo4j_database) as session:
        # Get all nodes
        nodes_result = await session.run("""
            MATCH (n)
            RETURN labels(n) as labels, properties(n) as props
            LIMIT 10
        """)
        
        print("📍 Nodes in Graph:")
        async for record in nodes_result:
            print(f"   - {record['labels']}: {record['props']}")
        
        # Get all relationships
        rels_result = await session.run("""
            MATCH (a)-[r]->(b)
            RETURN type(r) as rel_type, properties(r) as props
            LIMIT 10
        """)
        
        print("\n🔗 Relationships in Graph:")
        async for record in rels_result:
            print(f"   - {record['rel_type']}: {record['props']}")
    
    await driver.close()

await visualize_graph()

## 10. Summary and Next Steps

### What We Demonstrated:
1. ✅ Lapa LLM integration via vLLM with structured outputs
2. ✅ Graphiti knowledge graph for long-term memory
3. ✅ LangGraph agent orchestration with state management
4. ✅ Memory retrieval and contextual responses
5. ✅ Graph visualization and querying

### Key Features:
- **Temporal Memory**: Graphiti tracks when information was learned
- **Semantic Search**: Hybrid search (embeddings + BM25 + graph traversal)
- **Context Awareness**: Agent uses retrieved memories to personalize responses
- **Ukrainian Support**: Lapa LLM optimized for Ukrainian language

### Next Steps:
1. Add more conversations to build richer memory
2. Experiment with different query types
3. Visualize graph in Neo4j Browser (http://localhost:7474)
4. Test with multiple users/sessions
5. Implement memory cleanup strategies for old data

## 11. Cleanup (Optional)

In [ ]:
# Uncomment to clear all graph data
# from neo4j import AsyncGraphDatabase
#
# async def clear_graph():
#     driver = AsyncGraphDatabase.driver(
#         settings.neo4j_uri,
#         auth=(settings.neo4j_user, settings.neo4j_password)
#     )
#     async with driver.session(database=settings.neo4j_database) as session:
#         await session.run("MATCH (n) DETACH DELETE n")
#     await driver.close()
#     print("✅ Graph cleared")
#
# await clear_graph()